Ucitavamo dataset

In [1]:
import torch
import numpy as np 
from torch.utils.data import TensorDataset
from sklearn.preprocessing import OneHotEncoder

ulaz= np.loadtxt('Dataset.txt')
podaci=ulaz[:,1:]
#print(podaci)
b=np.max(podaci,axis=0)
podaci=podaci/b
#print(podaci)
#print(np.max(podaci),np.min(podaci))
obelezja=ulaz[:,:1]
#print(obelezja)
podaci=torch.Tensor(podaci)
obelezja=torch.Tensor(obelezja)

dataset=TensorDataset(podaci,obelezja)
train_set,test_set = torch.utils.data.random_split(dataset,[0.9,0.1])

print(len(test_set))
print(test_set[0])
print(test_set[0][0][2])

12500
(tensor([0.3936, 0.4364, 0.4423,  ..., 0.0000, 0.0000, 0.0899]), tensor([22.]))
tensor(0.4423)


Inicijalizacija mreze

In [2]:
import torch.nn as nn

class Mreza (nn.Module):
    def __init__ (self):
        super().__init__()
        self.act=nn.Sequential(
            nn.Linear(1564,300),
            nn.ReLU(),
            nn.Linear(300,120),
            nn.ReLU(),
            nn.Linear(120,50),
            nn.ReLU()
        )

    def forward (self, x):
        return self.act(x)
model = Mreza()

Podesavanje uredjaja

In [3]:
device="cpu"

Testiranje

In [4]:
def totensor(y):
    ret=np.zeros((len(y),50))
   # print(y[0])
   # print(y[0][0])
   # print(y[0][0].item())
    for i in range(0,len(y)):
        ret[i][round(y[i][0].item())]=1
    return torch.Tensor(ret)

def get_accuracy(y_hat,y):
    #print(y_hat,y)
    y_hat = y_hat.detach().numpy()
    y=y.detach().numpy()
    y_hat=np.argmax(y_hat,axis=0)
    corrects=(y_hat==y).sum()
    accuracy=100.0*corrects/y_hat.shape[0]
    return accuracy.item()

def test(model,sample):
    rez=model(sample)
    #print(rez)
    rez=rez.detach().numpy()
    #print(rez)
    poz=0
    for i in range(1,len(rez)):
        if(rez[i]>rez[poz]):
            poz=i
    return poz

def validation(model):
    model.to("cpu")
    kol=0
    for i in range(0,len(test_set)):
        out=test(model,test_set[i][0])
        ans=test_set[i][1][0].item()
        if round(ans)==out:
            kol+=1
    return kol

Treniranje

In [5]:
broj_epoha = 50
batch_velicina=100
loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_velicina,shuffle=True,num_workers=0,drop_last=True,pin_memory=True)
loss_funk = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

for epoha in range(1,broj_epoha+1):
    train_running_loss =0.0
    train_acc=0.0
    model = model.train()

    for batch_num, (x, y) in enumerate(loader):
            x = x.to(device)
            y = y.to(device)
            #print(y)
            y_hat = model(x)
            loss = loss_funk(y_hat, totensor(y))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)
            del y_hat

        
    epoch_loss = train_running_loss / batch_num
    epoch_acc = train_acc / batch_num
        
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f | Validation Accuracy: %.2f' \
          %(epoha, epoch_loss, epoch_acc, validation(model)/len(test_set)*100))

Epoch: 1 | Loss: 3.7698 | Train Accuracy: 155.66 | Validation Accuracy: 11.37
Epoch: 2 | Loss: 3.6381 | Train Accuracy: 155.19 | Validation Accuracy: 13.63
Epoch: 3 | Loss: 3.5728 | Train Accuracy: 154.24 | Validation Accuracy: 14.94
Epoch: 4 | Loss: 3.5265 | Train Accuracy: 154.55 | Validation Accuracy: 15.64
Epoch: 5 | Loss: 3.4945 | Train Accuracy: 154.33 | Validation Accuracy: 15.56
Epoch: 6 | Loss: 3.4659 | Train Accuracy: 153.59 | Validation Accuracy: 16.38
Epoch: 7 | Loss: 3.4427 | Train Accuracy: 154.46 | Validation Accuracy: 16.73
Epoch: 8 | Loss: 3.4232 | Train Accuracy: 154.64 | Validation Accuracy: 17.29
Epoch: 9 | Loss: 3.4042 | Train Accuracy: 154.25 | Validation Accuracy: 16.88
Epoch: 10 | Loss: 3.3876 | Train Accuracy: 154.52 | Validation Accuracy: 17.17
Epoch: 11 | Loss: 3.3739 | Train Accuracy: 154.35 | Validation Accuracy: 17.14
Epoch: 12 | Loss: 3.3561 | Train Accuracy: 154.79 | Validation Accuracy: 17.16
Epoch: 13 | Loss: 3.3430 | Train Accuracy: 153.80 | Validatio

In [83]:
print(validation(model))
for i in range(0,len(test_set)):
    if(test(model,test_set[i][0])==test_set[i][1][0].item()):
        print(i)
    #print(test(model,test_set[i][0]),test_set[i][1][0].item())

794
45
47
57
95
120
135
137
138
158
200
210
215
224
248
252
262
278
332
364
408
411
424
427
449
456
460
470
479
484
487
488
492
542
545
554
570
575
613
616
624
644
655
656
685
694
704
706
709
716
728
735
758
762
765
786
822
834
847
849
858
868
884
901
909
912
922
925
945
959
966
982
991
998
999
1025
1026
1065
1067
1089
1090
1095
1107
1111
1118
1120
1143
1147
1151
1226
1228
1273
1277
1279
1287
1292
1300
1301
1307
1316
1337
1342
1368
1373
1387
1436
1446
1462
1511
1524
1534
1537
1546
1616
1650
1668
1703
1708
1713
1761
1762
1781
1784
1791
1813
1824
1825
1828
1875
1881
1893
1896
1900
1917
1930
1960
1976
1993
1999
2011
2012
2016
2025
2039
2049
2075
2089
2112
2118
2134
2177
2197
2243
2281
2288
2293
2334
2341
2349
2374
2394
2396
2407
2459
2478
2482
2494
2502
2534
2539
2540
2557
2561
2572
2593
2601
2617
2633
2640
2675
2739
2755
2770
2791
2808
2814
2824
2827
2845
2850
2863
2888
2909
2916
2954
2985
3015
3019
3023
3071
3082
3085
3103
3134
3187
3197
3208
3215
3226
3232
3237
3260
3262
3267
3286
3316

In [88]:
print(get_accuracy(torch.tensor([[0,1,3],[0,5,3],[5,1,3]]),torch.tensor([2,2,1])))

tensor([[0, 1, 3],
        [0, 5, 3],
        [5, 1, 3]]) tensor([2, 2, 1])


AttributeError: 'numpy.ndarray' object has no attribute 'index'